## Dialog Management
___

Dialogue management system is responsible of predicting how an assistant should respond based on the state of the conversation as well as context. To train a dialogue management model you will need some training stories, domain file and model configuration. You can also create custom actions which, when predicted, can call an API, retrieve data from the database or perform some other integrations.


We have taught our chatbot how to understand user inputs. Now, it's time to teach our chatbot how to make responses by training a dialogue management model using Rasa Core.

*<strong>Before you start writing the code, please run this command so that you can asynchronous Rasa code in Jupyter Notebooks since Jupyter Notebooks already run on event loops.</strong>*


In [49]:
import nest_asyncio

nest_asyncio.apply()
print("Event loop ready.")

Event loop ready.


### Step 1: Writing Stories

The training data for dialogue management models is called stories. A story is an actual conversation where user inputs are expressed as intents as well as corresponding entities, and chatbot responses are expressed as actions.

Let's take a look into the format of the stories in more detail:

A story starts with ## and you can give it a name. Lines that start with * are messages sent by the user. Although you don't write the actual message, but rather the intent (and the entities) that represent what the user means. Lines that start with - are actions taken by your bot. In this case all of our actions are just messages sent back to the user, like utter_greet, but in general an action can do anything, including calling an API and interacting with the outside world.

In [27]:
stories_md = """
## happy path               <!-- name of the story - just for debugging -->
* greet              
  - utter_greet
* mood_great               <!-- user utterance, in format intent[entities] -->
  - utter_happy
* mood_affirm
  - utter_happy
* mood_affirm
  - utter_goodbye
  
## sad path 1               <!-- this is already the start of the next story -->
* greet
  - utter_greet             <!-- action the bot should execute -->
* mood_unhappy
  - utter_ask_picture
* inform{"animal":"dog"}  
  - action_retrieve_image
  - utter_did_that_help
* mood_affirm
  - utter_happy

## sad path 2
* greet
  - utter_greet
* mood_unhappy
  - utter_ask_picture
* inform{"group":"cat"}
  - action_retrieve_image
  - utter_did_that_help
* mood_deny
  - utter_goodbye
  
## sad path 3
* greet
  - utter_greet
* mood_unhappy{"group":"puppy"}
  - action_retrieve_image
  - utter_did_that_help
* mood_affirm
  - utter_happy
  
## strange user
* mood_affirm
  - utter_happy
* mood_affirm
  - utter_unclear

## say goodbye
* goodbye
  - utter_goodbye

## fallback
- utter_unclear

"""

%store stories_md > data/stories.md

Writing 'stories_md' (str) to file 'data/stories.md'.


### Step 2: Defining a Domain
The domain specifies the universe that the bot operates in. In chatbot's world this universe consists of intents and entities as well as the actions which appear in training stories. The domain can also contain the templates for the answers a chabot should use to respond to the user and slots which will help the chatbot to keep track of the context. Let's look into the domain of our bot:

In [50]:
domain_yml = """
intents:
- greet
- goodbye
- mood_affirm
- mood_deny
- mood_great
- mood_unhappy
- inform

slots:
  group:
    type: text
    
entities:
- group

actions:
- utter_greet
- utter_did_that_help
- utter_happy
- utter_goodbye
- utter_unclear
- utter_ask_picture
- action_retrieve_image

templates:
  utter_greet:
  - text: "Hey! How are you?"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_unclear:
  - text: "I am not sure what you are aiming for."
  
  utter_happy:
  - text: "Great carry on!"

  utter_goodbye:
  - text: "Bye"
  
  utter_ask_picture:
  - text: "To cheer you up, I can show you a cute picture of a dog, cat or a bird. Which one do you choose?"
"""

%store domain_yml > domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


### Step 3: Adding Custom Actions

The responses of the chatbot can be more than just simple text responses - we can call an API to retrieve some data which can later be used to create a response to user input. Let's create a custom action for our bot which, when predicted, will make an API and retrieve a picture of a dog, a cat or a bird, depending on which was specified by the user. The bot will know which type of picture should be received by retrieving the value of the slot group.

In [42]:
actions_py="""
from rasa.core.actions import Action
from rasa.core.events import SlotSet
from IPython.core.display import Image, display

import requests

class ApiAction(Action):
    def name(self):
        return "action_retrieve_image"

    def run(self, dispatcher, tracker, domain):
        
        group = tracker.get_slot('group')
        
        r = requests.get('https://randomfox.ca//images//108.jpg'.format(group))
        response = r.content.decode()
        response = response.replace('["',"")
        response = response.replace('"]',"")
   
        
        #display(Image(response[0], height=550, width=520))
        dispatcher.utter_message("Here is something to cheer you up: {}".format(response))
"""
%store actions_py > actions.py

Writing 'actions_py' (str) to file 'actions.py'.


### Step 4: Adding Policies

the Policies decides which action to take at every step in the conversation.We can specify what policies should be used to train it - in this case, the model is a neural network implemented in Keras which learns to predict which action to take next. We can also tweak the parameters of what percentage of training examples should be used for validation and how many epochs should be used for training.

So lets update the **config.yml** file which we had created for NLU

In [34]:
configuration = """
language: "en"

pipeline:
- name: "WhitespaceTokenizer"       # splits the sentence into tokens           
- name: "CRFEntityExtractor"                   # uses the pretrained spacy NER model
- name: "CountVectorsFeaturizer"     # transform the sentence into a vector representation
- name: "EmbeddingIntentClassifier"   # intent classifier

policies:
  - name: "KerasPolicy"
    featurizer:
    - name: MaxHistoryTrackerFeaturizer
      max_history: 5
      state_featurizer:
        - name: BinarySingleStateFeaturizer
  - name: "MemoizationPolicy"
    max_history: 5
  - name: "FallbackPolicy"
    nlu_threshold: 0.4
    core_threshold: 0.3
""" 

%store configuration > config.yml

Writing 'configuration' (str) to file 'config.yml'.


### Step 5: Training your Dialogue Model
Now we are good to train the dialogue management model. 

In [39]:
import rasa

model_path = rasa.train('domain.yml', 'config.yml','data/' ,'./models/current')
print(model_path)

### Step 6: Talking to the Bot

In [45]:
from rasa.jupyter import chat
chat(model_path)